# Deep learning framework example: Named Entity Recognition

This notebook demonstrates how to use the deeplearning API to train and test the model on the [CoNNL 2003 dataset](https://www.clips.uantwerpen.be/conll2003/ner/).  The task is to identify named entities (i.e. people, organizations etc).  See the [saved version](https://htmlpreview.github.io/?https://github.com/plandes/deepnlp/blob/master/example/ner/notebook/ner.html) for output.

**Important**: Please see the [Movie Review notebook example](https://github.com/plandes/deepnlp/blob/master/example/movie/notebook/movie.ipynb) for a more simple example of how to utilize the framework to tune pamateres in Jupyter notebooks.

In [ ]:
# environemnt configuration and set up: add this (deepnlp) library to the Python path and framework entry point
from mngfac import JupyterManagerFactory
fac = JupyterManagerFactory()
mng = fac()

## Print information about 

Use the factory to create the model executor.  The `write` method gives statistics on the data set that is configured on the executor.  Note that the first time this runs, the framework automatically downloads the corpus, vectorizers and creates batches for quick experimentation.

In [ ]:
facade = mng.create_facade()
mng.write()

## Tune hyperparameters

Set model parameters to get a feel for where they need to be before changing features.  Start with Glove 50 dimensional word embeddings with a learning rate of 0.01 and 8 epochs.

In [ ]:
facade.learning_rate = 0.01
facade.epochs = 8
mng.run()
facade.persist_result()

# Glove 300 embeddings

Next we use the same learning rate, but switch to the 300 dimension version of the embeddings.  The number of epochs is reduced because I have run the test before I know at what epoch the validation loss converges.  Since the model is saved only when the validation loss decreases, we early stop at 6 epochs.

In [ ]:
facade.epochs = 6
facade.embedding = 'glove_300_embedding'
mng.run()
facade.persist_result()

# word2vec Embeddings

Now we switch to the Google 300D word2vec pretrained vectors using 12 epochs, even though it has converged at 9 epochs previously.

In [ ]:
facade.embedding = 'word2vec_300_embedding'
mng.run()
facade.persist_result()

# BERT Embeddings

Now we test with Bert context aware frozen (not trainable) embeddings using 10 epochs.  We must empty the `net_settings` attributes, which are the lingustic features, since Bert tokenizes using the word piece algorithm and the tensor shapes will not align.  We'll address this later.

In [ ]:
facade = mng.create_facade('transformer')
mng.run()
facade.persist_result()

In [ ]:
facade = mng.create_facade('transformer', model_id='roberta-base')
mng.run()
facade.persist_result()

In [ ]:
facade = mng.create_facade('transformer', model_id='bert-large-cased')
facade.epochs = 8
mng.run()
facade.persist_result()

In [ ]:
import logging
logging.getLogger('zensols.deepnlp.transformer.optimizer').setLevel(logging.INFO)
mng.config('model_settings',
           scheduler_class_name='zensols.deepnlp.transformer.TransformerSchedulerFactory',
           scheduler_params="dict: {'name': 'linear', 'num_warmup_steps': 0.01}")
facade = mng.create_facade('transformer', model_id='bert-large-cased')
facade.epochs = 22
mng.run()
facade.persist_result()

# Compare results

Generate a dataframe with the performance metrics of the previous run

In [ ]:
from zensols.deeplearn.result import ModelResultManager, ModelResultReporter
rm: ModelResultManager = facade.result_manager
reporter = ModelResultReporter(rm, include_validation=False)
reporter.dataframe.drop(columns=['name'])